In [ ]:
!pip install -qq datasets coqui-TTS
!pip install -Uqq g2p_id_py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 284.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 65.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 81.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 81.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
MAIN_DIR = "/content/drive/MyDrive/dataspoofv2"
train_df = pd.read_csv(os.path.join(MAIN_DIR, "indspeech_news_lvcsr_train_val.csv"))
test_df = pd.read_csv(os.path.join(MAIN_DIR, "indspeech_news_lvcsr_test.csv"))

In [ ]:
train_df.head()

,file_id,spk_id,text,audio_file,audio_duration
0,ID_T_Ind058_M_1973,ID_Ind058,Kami akan tetap di sini sampai Eurico dibebaskan.,/root/.cache/huggingface/datasets/downloads/ex...,4.34
1,ID_T_Ind073_F_2412,ID_Ind073,Akbar mengiyakan.,/root/.cache/huggingface/datasets/downloads/ex...,2.68
2,ID_T_Ind358_F_1940,ID_Ind358,Salah satunya dialami oleh pabrik sepatu merek...,/root/.cache/huggingface/datasets/downloads/ex...,4.49
3,ID_T_Ind161_F_2021,ID_Ind161,Sebagai ungkapan syukur.,/root/.cache/huggingface/datasets/downloads/ex...,2.65
4,ID_T_Ind152_M_1710,ID_Ind152,Bukan perang etnis.,/root/.cache/huggingface/datasets/downloads/ex...,2.32


In [ ]:
# CONVERT GRAPHEME TO PHONEME
from g2p_id import G2p

def grapheme_to_phomene(dataset):
  converter = G2p()
  dataset['phoneme'] = dataset['text'].apply(lambda x: converter(x))
  for idx, row in dataset.iterrows():

    seq_phoneme = []
    for phoneme_ls in row['phoneme']:
      seq_phoneme.append(''.join(phoneme_ls))

    dataset.at[idx, 'phoneme'] = ' '.join(seq_phoneme)

  return dataset

df = grapheme_to_phomene(train_df)
test_df = grapheme_to_phomene(test_df)

train_df[['text', 'phoneme']].head()

,text,phoneme
0,Kami akan tetap di sini sampai Eurico dibebaskan.,kami akan tətap di sini sampaʔi eʔuritʃo dibeb...
1,Akbar mengiyakan.,akbar məŋijakan .
2,Salah satunya dialami oleh pabrik sepatu merek...,salah satuɲa diʔalami oleh pabrik səpatu merək...
3,Sebagai ungkapan syukur.,səbaɡaʔi uŋkapan ʃukur .
4,Bukan perang etnis.,bukan pəraŋ etnis .


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_spoofed
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_spoofed/flac
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_spoofed
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_spoofed/flac

In [ ]:
import torch
from TTS.api import TTS

# UPLOAD speakers.pth
device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS(
    model_path="/content/drive/MyDrive/idasvspoofing-project/tts_model/checkpoint_1260000-inference.pth",
    config_path="/content/drive/MyDrive/idasvspoofing-project/tts_model/config.json"
).to(device)

In [ ]:
# Generate speech inference
tts.tts_to_file(
    text="kata ʃatʃhrial .",
    file_path="output1.flac",
    speaker="gadis"
)

In [ ]:
# Split dataframe to train and validation dataframes
train_mask = df['file_id'].str.contains('T')
val_mask = df['file_id'].str.contains('V')

val_df = df[val_mask]
train_df = df[train_mask]

In [ ]:
len(train_df), len(val_df)

(31680, 7920)

In [ ]:
male_df = train_df[train_df['file_id'].str.contains('M')].index

protocol_lines = []
counter = 0
for index, row in train_df.iloc[male_df].iterrows():
  if counter == 2500:
    break

  line = f"ID_Ind404 ID_T_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_spoofed/flac/ID_T_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="wibowo" # can use ardi for alt male voice
  )
  protocol_lines.append(line)

  counter += 1

female_df = train_df[train_df['file_id'].str.contains('F')].index
counter = 0
for index, row in train_df.iloc[female_df].iterrows():
  if counter == 2500:
    break

  line = f"ID_Ind505 ID_T_ID_Ind404_F_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_spoofed/flac/ID_T_ID_Ind505_F_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="gadis"
  )
  protocol_lines.append(line)

  counter += 1

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.train.spoofed.txt', 'w') as f:
  f.write('\n'.join(protocol_lines))

In [ ]:
val_df = val_df.reset_index(drop=True)

male_df = val_df[val_df['file_id'].str.contains('M')].index

protocol_lines = []
counter = 0
for index, row in val_df.iloc[male_df].iterrows():
  if counter == 1500:
    break

  line = f"ID_Ind404 ID_V_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_spoofed/flac/ID_V_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="wibowo" # can use ardi for alt male voice
  )
  protocol_lines.append(line)

  counter += 1

female_df = val_df[val_df['file_id'].str.contains('F')].index
counter = 0
for index, row in val_df.iloc[female_df].iterrows():
  if counter == 1500:
    break

  line = f"ID_Ind505 ID_V_ID_Ind404_F_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_spoofed/flac/ID_V_ID_Ind505_F_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="gadis"
  )
  protocol_lines.append(line)

  counter += 1

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.val.spoofed.txt', 'w') as f:
  f.write('\n'.join(protocol_lines))

In [ ]:
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_spoofed
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_spoofed/flac

In [ ]:
male_df = test_df[test_df['file_id'].str.contains('M')].index

protocol_lines = []
counter = 0
for index, row in test_df.iloc[male_df].iterrows():
  if counter == 1000:
    break

  line = f"ID_Ind404 ID_TT_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_spoofed/flac/ID_TT_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="wibowo" # can use ardi for alt male voice
  )
  protocol_lines.append(line)

  counter += 1

female_df = test_df[test_df['file_id'].str.contains('F')].index
counter = 0
for index, row in test_df.iloc[female_df].iterrows():
  if counter == 1000:
    break

  line = f"ID_Ind505 ID_TT_ID_Ind404_F_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_spoofed/flac/ID_TT_ID_Ind505_F_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="gadis"
  )
  protocol_lines.append(line)

  counter += 1

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.spoofed.txt', 'w') as f:
  f.write('\n'.join(protocol_lines))

In [ ]:
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_joined
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_joined/flac
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_joined
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_joined/flac
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_joined
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_joined/flac

In [ ]:
import os
import shutil

def join_directories(source_dir1, source_dir2, dest_dir):
  """Joins the contents of two directories into a destination directory.

  Args:
    source_dir1: Path to the first source directory.
    source_dir2: Path to the second source directory.
    dest_dir: Path to the destination directory.
  """
  if not os.path.exists(dest_dir):
      os.makedirs(dest_dir)

  for filename in os.listdir(source_dir1):
    source_path = os.path.join(source_dir1, filename)
    dest_path = os.path.join(dest_dir, filename)
    if os.path.isfile(source_path):
        shutil.copy2(source_path, dest_path)  # copy2 preserves metadata

  for filename in os.listdir(source_dir2):
    source_path = os.path.join(source_dir2, filename)
    dest_path = os.path.join(dest_dir, filename)
    if os.path.isfile(source_path) and not os.path.exists(dest_path): #prevent overwriting files from dir1
        shutil.copy2(source_path, dest_path)

source_train1 = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_spoofed/flac"
source_train2 = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_bonafide/flac"
dest_dir_train = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_joined/flac"
join_directories(source_train1, source_train2, dest_dir_train)

source_val1 = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_spoofed/flac"
source_val2 = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_bonafide/flac"
dest_dir_val = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_joined/flac"
join_directories(source_val1, source_val2, dest_dir_val)

source_test1 = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_spoofed/flac"
source_test2 = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_bonafide/flac"
dest_dir_test = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_joined/flac"
join_directories(source_test1, source_test2, dest_dir_test)

NameError: name 'source_test2' is not defined

In [ ]:
source_test1 = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_spoofed/flac"
source_test2 = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_bonafide/flac"
dest_dir_test = "/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_joined/flac"
join_directories(source_test1, source_test2, dest_dir_test)

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_joined/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_joined/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_joined/flac')))

10000
6000
4000


In [ ]:
!zip -r IDASVspoofing_train_spoof.zip /content/joined_audio

from google.colab import files
files.download("/content/IDASVspoofing_train_spoof.zip")
files.download("/content/IDASVspoofing.cm.train.txt.spoofed")

Streaming output truncated to the last 5000 lines.
  adding: content/joined_audio/ID_T_ID_Ind505_F_01992_src-Ind019.flac (deflated 31%)
  adding: content/joined_audio/ID_T_ID_Ind404_M_02506_src-Ind023.flac (deflated 48%)
  adding: content/joined_audio/ID_T_ID_Ind404_M_03311_src-Ind031.flac (deflated 37%)
  adding: content/joined_audio/ID_T_ID_Ind505_F_02877_src-Ind027.flac (deflated 51%)
  adding: content/joined_audio/ID_T_ID_Ind404_M_05722_src-Ind053.flac (deflated 59%)
  adding: content/joined_audio/ID_T_ID_Ind505_F_07201_src-Ind066.flac (deflated 36%)
  adding: content/joined_audio/ID_T_ID_Ind505_F_03136_src-Ind029.flac (deflated 68%)
  adding: content/joined_audio/ID_T_ID_Ind404_M_02328_src-Ind022.flac (deflated 46%)
  adding: content/joined_audio/ID_T_ID_Ind404_M_05333_src-Ind049.flac (deflated 64%)
  adding: content/joined_audio/ID_T_ID_Ind404_M_02463_src-Ind023.flac (deflated 49%)
  adding: content/joined_audio/ID_T_ID_Ind505_F_00107_src-Ind001.flac (deflated 44%)
  adding: cont

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
male_df = train_df[train_df['file_id'].str.contains('M')].index

protocol_lines = []
counter = 0
for index, row in train_df.iloc[male_df[5000:]].iterrows():
  if counter == 2500:
    break

  line = f"ID_Ind404 ID_V_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/val_spoof/audio_male/flac/ID_V_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="wibowo" # can use ardi for alt male voice
  )
  protocol_lines.append(line)

  counter += 1

female_df = train_df[train_df['file_id'].str.contains('F')].index
counter = 0
for index, row in train_df.iloc[female_df[5000:]].iterrows():
  if counter == 2500:
    break

  line = f"ID_Ind505 ID_V_ID_Ind505_F_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/val_spoof/audio_female/flac/ID_V_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="gadis"
  )
  protocol_lines.append(line)

  counter += 1

with open('/content/IDASVspoofing.cm.val.txt.spoof', 'w') as f:
  f.write('\n'.join(protocol_lines))

In [ ]:
# Example usage (replace with your actual directory paths)
source_dir1 = '/content/val_spoof/audio_female/flac'
source_dir2 = '/content/val_spoof/audio_male/flac'
dest_dir = '/content/joined_audio_val'
join_directories(source_dir1, source_dir2, dest_dir)

!zip -r IDASVspoofing_val_spoof.zip /content/joined_audio_val

from google.colab import files
files.download('/content/IDASVspoofing.cm.val.txt.spoof')
files.download("/content/IDASVspoofing_val_spoof.zip")

In [ ]:
!mkdir /content/test_spoof
!mkdir /content/test_spoof/audio_male/
!mkdir /content/test_spoof/audio_female/
!mkdir /content/test_spoof/audio_male/flac
!mkdir /content/test_spoof/audio_female/flac

In [ ]:
male_df = test_df[test_df['file_id'].str.contains('M')].index

protocol_lines = []
counter = 0
for index, row in test_df.iloc[male_df[1000:]].iterrows():
  if counter == 250:
    break

  line = f"ID_Ind404 ID_TT_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/test_spoof/audio_male/flac/ID_TT_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="wibowo" # can use ardi for alt male voice
  )
  protocol_lines.append(line)

  counter += 1

female_df = test_df[test_df['file_id'].str.contains('F')].index
counter = 0
for index, row in test_df.iloc[female_df[1000:]].iterrows():
  if counter == 250:
    break

  line = f"ID_Ind505 ID_TT_ID_Ind505_F_{index:05}_src-{row['spk_id'][3:]} - A10 spoof"
  tts.tts_to_file(
      text=row['phoneme'],
      file_path=f"/content/test_spoof/audio_female/flac/ID_TT_ID_Ind404_M_{index:05}_src-{row['spk_id'][3:]}.flac",
      speaker="gadis"
  )
  protocol_lines.append(line)

  counter += 1

with open('/content/IDASVspoofing.cm.test.txt.spoof', 'w') as f:
  f.write('\n'.join(protocol_lines))

# Dataset Concatenation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
os.listdir("/content/drive/MyDrive/dataspoofv2/")

['indspeech_news_lvcsr_train_val.csv',
 'IDASVspoofing_train_bonafide',
 'IDASVspoofing_val_bonafide',
 'IDASVspoofing.cm.train.bonafide.txt',
 'IDASVspoofing.cm.val.bonafide.txt',
 'indspeech_news_lvcsr_test.csv',
 'IDASVspoofing_test_bonafide',
 'IDASVspoofing.cm.test.bonafide.txt',
 'trainval_bonafide_shuffle.csv',
 'IDASVspoofing_train_spoofed',
 'IDASVspoofing_val_spoofed',
 'IDASVspoofing.cm.train.spoofed.txt',
 '.ipynb_checkpoints',
 'IDASVspoofing.cm.val.spoofed.txt',
 'IDASVspoofing_test_spoofed',
 'IDASVspoofing.cm.test.spoofed.txt',
 'IDASVspoofing_train_joined',
 'IDASVspoofing_val_joined',
 'IDASVspoofing_test_joined']

In [ ]:
!cat /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.train.bonafide.txt /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.train.spoofed.txt > /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.train.joined.txt
!cat /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.val.bonafide.txt /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.val.spoofed.txt > /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.val.joined.txt
!cat /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.bonafide.txt /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.spoofed.txt > /content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.joined.txt

In [ ]:
def check_lines(small_file, big_file):
   # Read the bigger file into a set for efficient lookup
   with open(big_file, 'r') as f:
       big_lines = set(line.strip() for line in f)

   # Check each line in small file
   with open(small_file, 'r') as f:
       for line in f:
           line = line.strip()
           if line not in big_lines:
               print(f"Missing line: {line}")

# Usage
check_lines('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.bonafide.txt', '/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.joined.txt')
check_lines('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.spoofed.txt', '/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.joined.txt')

Missing line: ID_Ind390 ID_TT_Ind390_F_2938 - - bonafide
Missing line: ID_Ind404 ID_TT_ID_Ind404_M_00000_src-Ind239 - A10 spoof


In [ ]:
with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.train.joined.txt', 'r') as file:
   line_count = sum(1 for line in file)
print(f"Number of lines: {line_count}")

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.val.joined.txt', 'r') as file:
   line_count = sum(1 for line in file)
print(f"Number of lines: {line_count}")

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.joined.txt', 'r') as file:
   line_count = sum(1 for line in file)
print(f"Number of lines: {line_count}")

Number of lines: 9999
Number of lines: 5999
Number of lines: 3999


In [ ]:
# !mv /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_spoof/*.flac /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_spoof/flac/

In [ ]:
!mkdir /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train_joined
!mkdir /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train_joined/flac
!cp /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train/flac/* /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train_joined/flac && cp /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train_spoof/flac/* /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train_joined/flac

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train_spoof/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_train_joined/flac')))

import re

protocol_lines = []
pattern = r"Ind(\d{3})"
for files in os.listdir('/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_bonafid/flac'):
  match = re.search(pattern, files)
  if match:
      ind_number = match.group()

  line = f"{ind_number} {files} - - bonafide"
  protocol_lines.append(line)

for files in os.listdir('/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_spoof/flac'):
  match = re.search(pattern, files)
  if match:
      ind_number = match.group()

  line = f"{ind_number} {files} - A10 spoof"
  protocol_lines.append(line)

with open('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing.cm.train.txt.joined', 'w') as f:
  f.write('\n'.join(protocol_lines))
with open('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing.cm.train.txt.joined') as f:
    print(len(f.readlines()))

10000
10000
20000
20000


In [ ]:
!mkdir /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_joined
!mkdir /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_joined/flac
!cp /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val/flac/* /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_joined/flac && cp /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_spoof/flac/* /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_joined/flac

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_spoof/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_joined/flac')))

import re

protocol_lines = []
pattern = r"Ind(\d{3})"
for files in os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val/flac'):
  match = re.search(pattern, files)
  if match:
      ind_number = match.group()

  line = f"{ind_number} {files} - - bonafide"
  protocol_lines.append(line)

for files in os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_val_spoof/flac'):
  match = re.search(pattern, files)
  if match:
      ind_number = match.group()

  line = f"{ind_number} {files} - A10 spoof"
  protocol_lines.append(line)

with open('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing.cm.val.txt.joined', 'w') as f:
  f.write('\n'.join(protocol_lines))
with open('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing.cm.val.txt.joined') as f:
    print(len(f.readlines()))

2500
5000
7500
7500


In [ ]:
!mkdir /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_joined
!mkdir /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_joined/flac
!cp /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test/flac/* /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_joined/flac && cp /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_spoof/* /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_joined/flac

cp: cannot stat '/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_spoof/flac/*': No such file or directory


In [ ]:
!cp /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_spoof/* /content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_joined/flac

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_spoof')))
print(len(os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_joined/flac')))

import re

protocol_lines = []
pattern = r"Ind(\d{3})"
for files in os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test/flac'):
  match = re.search(pattern, files)
  if match:
      ind_number = match.group()

  line = f"{ind_number} {files} - - bonafide"
  protocol_lines.append(line)

for files in os.listdir('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing_test_spoof'):
  match = re.search(pattern, files)
  if match:
      ind_number = match.group()

  line = f"{ind_number} {files} - A10 spoof"
  protocol_lines.append(line)

with open('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing.cm.test.txt.joined', 'w') as f:
  f.write('\n'.join(protocol_lines))
with open('/content/drive/MyDrive/dataset-IDASVspoofing/IDASVspoofing.cm.test.txt.joined') as f:
    print(len(f.readlines()))

1999
500
2499
2499
